In [167]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [168]:

import time
import numpy as np
import cv2
from matplotlib import pyplot as plt
import os
import csv

from os import listdir
from os.path import isfile, join

In [169]:
train_data_list = []
with open('drive/My Drive/korat_sauce/img_csv.csv') as f_img:
    img_bins = csv.reader(f_img)
    for im in img_bins:
        train_data_list.append(im)
    f_img.close()

# r_bins =  np.array(train_data_list[0][1:33]).astype(np.float)
# g_bins = np.array(train_data_list[0][33:65]).astype(np.float)
# b_bins = np.array(train_data_list[0][65:97]).astype(np.float)




In [170]:
r_bins = []
g_bins = []
b_bins = []
target = []
data_for_train = []
for i in range(len(train_data_list)):
    img_data = np.array(train_data_list[i]).astype(np.float)
    min_x =  min(img_data) 
    max_x = max(img_data)
    for j in range(1,len(img_data)):
        img_data[j] = (img_data[j]-min_x)/(max_x-min_x) #normalization
    data_for_train.append(img_data)
    r_bins.append(img_data[1:33])
    g_bins.append(img_data[33:65])
    b_bins.append(img_data[65:97])
    target.append(img_data[0])


bins_order = np.arange(32).astype(np.float)
for i in range(len(bins_order)):
    bins_order[i]  = (bins_order[i]-1)/31

**CREATE WEIGHT**

In [171]:
w_32t16_R = np.random.rand(16,32).astype(np.float) 
w_32t16_G = np.random.rand(16,32).astype(np.float) 
w_32t16_B = np.random.rand(16,32).astype(np.float) 

In [172]:
w_16t4_R = np.random.rand(4,16).astype(np.float) 
w_16t4_G = np.random.rand(4,16).astype(np.float) 
w_16t4_B = np.random.rand(4,16).astype(np.float) 

**CREATE BIAS**

In [173]:
# b_32t16 = np.ones(32,16).astype(np.float)
# b_16t4 = np.ones(16,4).astype(np.float)
bias_R = np.ones(2)
bias_G = np.ones(2)
bias_B= np.ones(2)

**PERCEPTRON**

In [174]:
def perceptron(input_data,weight,bias):
    # print(np.dot(input_data,np.transpose(weight))+bias)
    return np.dot(input_data,np.transpose(weight))+bias

**TRANSFER FUNCTION**

In [175]:
def transfer_arctan(result):
    return np.arctan(result)

**TRAIN WEIGHT**

In [176]:
def tr_16t4(target,result,w_16t4,bias):
    target_array = 0
    if(target == 1):
          target_array = np.array([1.5,0.01,0.01,0.01]).astype(np.float)
    elif(target == 2):
          target_array = np.array([0.01,1.5,0.01,0.01]).astype(np.float)
    elif(target == 3):
          target_array = np.array([0.01,0.01,1.5,0.01]).astype(np.float)
    elif(target == 4):
          target_array = np.array([0.01,0.01,0.01,1.01]).astype(np.float)
    
    error = np.zeros(4)
    max_indx_target = np.where(target_array == max(target_array))
    #print("max1 = ",max_indx_target)
    max_indx_result = np.where(result == max(result))
    #print("max1 = ",max_indx_result)
    if (np.array_equal(max_indx_target,max_indx_result)):
        pass
    else:
        for i in range(len(error)):
            if (i == max_indx_target[0]):
                error[i] = abs(target_array[i]-result[i])
                #print("max")
            else:
                error[i] = 0-abs(target_array[i]-result[i])

    delta_weight = np.ones((4,16)).astype(np.float) 
    #print(error)
    for row in range(len(w_16t4)):
        for col in range(len(w_16t4[0])):
            delta_weight[row][col] = error[row]*w_16t4[row][col]/sum(w_16t4[row])
            
    
    w_16t4 = w_16t4+delta_weight
    bias[1] = bias[1]+sum(error)*0.01
    return w_16t4,bias,delta_weight

In [177]:
def tr_32t16(delta_weight,w_32t16,bias):
    sum_error = np.ones(16).astype(np.float)
    for i in range(len(delta_weight)):
        sum_error+=delta_weight[i]
    delta_weight_in = np.ones((16,32)).astype(np.float) 
    for row in range(len(w_32t16)):
        for col in range(len(w_32t16[0])):
            delta_weight_in[row][col] = sum_error[row]*w_32t16[row][col]/sum(w_32t16[row])
    w_32t16 = w_32t16+delta_weight_in
    bias[0] = bias[0]+sum(sum_error)*0.01
    return w_32t16,bias


**LEARN**

In [185]:
def lr(input_data,target,w_32t16,w_16t4,bias,learn_round):
    round_train = 0
    check_true = 0
    # list_check = []
    while  (check_true < int(len(input_data)*90/100) and round_train<learn_round):
        round_train+=1
        check_true = 0

        for pic_input_r in range(len(input_data)):
            percept_32t16 =  perceptron(input_data[pic_input_r],w_32t16,bias[0])
            trans_32t16  = transfer_arctan(percept_32t16)
            percept_16t4  = perceptron(trans_32t16,w_16t4,bias[1])
            trans_16t4  = transfer_arctan(percept_16t4)

            target_array = 0
            if(target[pic_input_r] == 1):
                  target_array = np.array([1.5,0.01,0.01,0.01]).astype(np.float)
            elif(target[pic_input_r] == 2):
                  target_array = np.array([0.01,1.5,0.01,0.01]).astype(np.float)
            elif(target[pic_input_r] == 3):
                  target_array = np.array([0.01,0.01,1.5,0.01]).astype(np.float)
            elif(target[pic_input_r] == 4):
                  target_array = np.array([0.01,0.01,0.01,1.01]).astype(np.float)
    
            
            max_indx_target = np.where(target_array == max(target_array))
            max_indx_result = np.where(trans_16t4 == max(trans_16t4))
            if(np.array_equal(max_indx_target,max_indx_result) ):
                check_true +=1
                # list_check.append([max_indx_target,trans_16t4])
                #print("true : ",check_true)
            while not(np.array_equal(max_indx_target,max_indx_result) ):
                  w_16t4,bias,delta_weight = tr_16t4(target[pic_input_r],trans_16t4,w_16t4,bias)
                  w_32t16,bias = tr_32t16(delta_weight,w_32t16,bias)


                  percept_32t16 =  perceptron(input_data[pic_input_r],w_32t16,bias[0])
                  trans_32t16  = transfer_arctan(percept_32t16)
                  percept_16t4  = perceptron(trans_32t16,w_16t4,bias[1])
                  trans_16t4  = transfer_arctan(percept_16t4)
                  #print('result = ',trans_16t4,'              real =',target_array)
                  max_indx_result = np.where(trans_16t4 == max(trans_16t4))
                  #break
            #break
        # return w_32t16,w_16t4,bias,list_check
    #     break
        print(round_train)
    print("Now,Weight are OK! and use round for train = ",round_train," and accuracy is ",check_true,"/",len(input_data))
    return w_32t16,w_16t4,bias,list_check

In [186]:
w_32t16_R,w_16t4_R,bias_R = lr(r_bins,target,w_32t16_R,w_16t4_R,bias_R,50)


[[(array([0]),), array([-1.5707898, -1.5707898, -1.5707898, -1.5707898])],
 [(array([0]),), array([-1.5707898, -1.5707898, -1.5707898, -1.5707898])],
 [(array([0]),), array([-1.5707898, -1.5707898, -1.5707898, -1.5707898])],
 [(array([1]),), array([-1.5707898, -1.5707898, -1.5707898, -1.5707898])],
 [(array([1]),), array([-1.5707898, -1.5707898, -1.5707898, -1.5707898])],
 [(array([2]),), array([-1.5707898, -1.5707898, -1.5707898, -1.5707898])],
 [(array([2]),), array([-1.5707898, -1.5707898, -1.5707898, -1.5707898])],
 [(array([2]),), array([-1.5707898, -1.5707898, -1.5707898, -1.5707898])],
 [(array([2]),), array([-1.5707898, -1.5707898, -1.5707898, -1.5707898])],
 [(array([2]),), array([-1.5707898, -1.5707898, -1.5707898, -1.5707898])],
 [(array([2]),), array([-1.5707898, -1.5707898, -1.5707898, -1.5707898])],
 [(array([2]),), array([-1.5707898, -1.5707898, -1.5707898, -1.5707898])],
 [(array([2]),), array([-1.5707898, -1.5707898, -1.5707898, -1.5707898])],
 [(array([1]),), array([-

In [187]:
w_32t16_G,w_16t4_G,bias_G = lr(g_bins,target,w_32t16_G,w_16t4_G,bias_G,50)

[[(array([0]),), array([-1.57078961, -1.57078961, -1.57078961, -1.57078961])],
 [(array([0]),), array([-1.57078961, -1.57078961, -1.57078961, -1.57078961])],
 [(array([1]),), array([-1.57078961, -1.57078961, -1.57078961, -1.57078961])],
 [(array([1]),), array([-1.57078961, -1.57078961, -1.57078961, -1.57078961])],
 [(array([1]),), array([-1.57078961, -1.57078961, -1.57078961, -1.57078961])],
 [(array([2]),), array([-1.57078961, -1.57078961, -1.57078961, -1.57078961])],
 [(array([2]),), array([-1.57078961, -1.57078961, -1.57078961, -1.57078961])],
 [(array([2]),), array([-1.57078961, -1.57078961, -1.57078961, -1.57078961])],
 [(array([2]),), array([-1.57078961, -1.57078961, -1.57078961, -1.57078961])],
 [(array([2]),), array([-1.57078961, -1.57078961, -1.57078961, -1.57078961])],
 [(array([2]),), array([-1.57078961, -1.57078961, -1.57078961, -1.57078961])],
 [(array([2]),), array([-1.57078961, -1.57078961, -1.57078961, -1.57078961])],
 [(array([2]),), array([-1.57078961, -1.57078961, -1

In [188]:
w_32t16_B,w_16t4_B,bias_B = lr(r_bins,target,w_32t16_B,w_16t4_B,bias_B,50)


[[(array([0]),), array([-1.57078958, -1.57078958, -1.57078958, -1.57078958])],
 [(array([0]),), array([-1.57078958, -1.57078958, -1.57078958, -1.57078958])],
 [(array([1]),), array([-1.57078958, -1.57078958, -1.57078958, -1.57078958])],
 [(array([1]),), array([-1.57078958, -1.57078958, -1.57078958, -1.57078958])],
 [(array([1]),), array([-1.57078958, -1.57078958, -1.57078958, -1.57078958])],
 [(array([2]),), array([-1.57078958, -1.57078958, -1.57078958, -1.57078958])],
 [(array([2]),), array([-1.57078958, -1.57078958, -1.57078958, -1.57078958])],
 [(array([2]),), array([-1.57078958, -1.57078958, -1.57078958, -1.57078958])],
 [(array([2]),), array([-1.57078958, -1.57078958, -1.57078958, -1.57078958])],
 [(array([2]),), array([-1.57078958, -1.57078958, -1.57078958, -1.57078958])],
 [(array([2]),), array([-1.57078958, -1.57078958, -1.57078958, -1.57078958])],
 [(array([2]),), array([-1.57078958, -1.57078958, -1.57078958, -1.57078958])],
 [(array([2]),), array([-1.57078958, -1.57078958, -1

In [189]:
 from google.colab import drive
drive.mount('/content/drive')

!unzip drive/"My Drive"/korat_sauce/test_all_sauce4.zip

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Archive:  drive/My Drive/korat_sauce/test_all_sauce4.zip
   creating: test_all_sauce/
  inflating: test_all_sauce/maeyupa.0.jpg  
  inflating: test_all_sauce/maeyupa.1.jpg  
  inflating: test_all_sauce/maeyupa.10.jpg  
  inflating: test_all_sauce/maeyupa.11.jpg  
  inflating: test_all_sauce/maeyupa.12.jpg  
  inflating: test_all_sauce/maeyupa.13.jpg  
  inflating: test_all_sauce/maeyupa.14.jpg  
  inflating: test_all_sauce/maeyupa.15.jpg  
  inflating: test_all_sauce/maeyupa.16.jpg  
  inflating: test_all_sauce/maeyupa.17.jpg  
  inflating: test_all_sauce/maeyupa.18.jpg  
  inflating: test_all_sauce/maeyupa.19.jpg  
  inflating: test_all_sauce/maeyupa.2.jpg  
  inflating: test_all_sauce/maeyupa.20.jpg  
  inflating: test_all_sauce/maeyupa.21.jpg  
  inflating: test_all_sauce/maeyupa.22.jpg  
  inflating: test_all_sauce/maeyupa.23.jpg  
  inflating: test_all_s

In [190]:
mypath_test = 'test_all_sauce/'
imagePaths_test = [mypath_test+f for f in listdir(mypath_test) if isfile(join(mypath_test, f))]

In [191]:
def histrogam_to_32_bins(image_file):
    img = cv2.imread(image_file)
    if(len(img[0])>len(img[0][0])):
        img = np.rot90(img)
    img = cv2.resize(img, (150,180))
    ##cv2.imshow("img",img)


    all_r_list = []
    all_g_list = []
    all_b_list = []

    for row in range(len(img)):
        for col in range(len(img[0])):
            all_r_list.append(img[row][col][0])
            all_g_list.append(img[row][col][1])
            all_b_list.append(img[row][col][2])

    ##r_list = np.unique(all_r_list,axis=0)
    ##g_list = np.unique(all_g_list,axis=0)
    ##b_list = np.unique(all_b_list,axis=0)

    r_list = np.zeros(256)
    g_list = np.zeros(256)
    b_list = np.zeros(256)


    for val in all_r_list:
        r_list[val] = r_list[val]+1
    for val in all_g_list:
        g_list[val] = g_list[val]+1
    for val in all_b_list:
        b_list[val] = b_list[val]+1

    r_bins = []
    g_bins = []
    b_bins = []
    
    for range_bins in range(256//8):
        
        r_bins.append(sum(r_list[range_bins*8:range_bins*8+8]))
        g_bins.append(sum(g_list[range_bins*8:range_bins*8+8]))
        b_bins.append(sum(b_list[range_bins*8:range_bins*8+8]))

    return r_bins,g_bins,b_bins

In [198]:
#######
target = []
r_bins_list = []
g_bins_list = []
b_bins_list = []
accuracy_pack_test = 0
for (i, img_path) in enumerate(imagePaths_test):
  class_of_test = img_path.split(".")[0].split("/")[1]

  r_bins,g_bins,b_bins = histrogam_to_32_bins(img_path)
  r_bins_list.append(np.array(r_bins))
  g_bins_list.append(np.array(g_bins))
  b_bins_list.append(np.array(b_bins))
  
  
  if(class_of_test=="maeyupa"):
    target.append(1)

  elif(class_of_test=="ocean"):
    target.append(2)

  elif(class_of_test=="zapnuer"):
    target.append(3)

  elif(class_of_test=="zapnuerchick"):
    target.append(4)



In [206]:
type(r_bins_list)

list

In [204]:
w_32t16_R,w_16t4_R,bias_R= lr(r_bins_list,target,w_32t16_R,w_16t4_R,bias_R,1)

ValueError: ignored

In [205]:
w_32t16_G,w_16t4_G,bias_G = lr(g_bins_list,target,w_32t16_G,w_16t4_G,bias_G,1)

ValueError: ignored

In [ ]:
w_32t16_B,w_16t4_B,bias_B = lr(b_bins_list,target,w_32t16_B,w_16t4_B,bias_B,1)